<a href="https://colab.research.google.com/github/leandrobarbieri/pydata-book/blob/2nd-edition/Juncao_Combinacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Junção e combinação

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.options.display.max_rows = 20
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Indexação hierarquica

In [ ]:
dados1 = pd.Series(np.random.randn(9), 
                   index=[["a", "a", "a", "b", "b", "b", "c", "c", "c"], [1, 2, 3, 1, 2, 3, 1, 2, 3]])

# index com dois níveis
dados1.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 2),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('c', 3)],
           )

In [ ]:
dados1

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   2    1.965781
   3    1.393406
c  1    0.092908
   2    0.281746
   3    0.769023
dtype: float64

In [ ]:
# uma busca indexada usando um nível externo retorna multiplas linhas
dados1["a"]

1   -0.204708
2    0.478943
3   -0.519439
dtype: float64

In [ ]:
# para retornar um item específico temos que usar os dois indices nos dois niveis
dados1["a"][1]

-0.20470765948471295

In [ ]:
# primeira linha do indice externo de "a" e "b"
# é como se o segundo nivel tivesse virado uma coluna de um df
dados1.loc[["a", "b"], 1]

a  1   -0.204708
b  1   -0.555730
dtype: float64

In [ ]:
# transforma o segundo nivel em colunas (pivot). Por padrão usa o nivel mais interno
dados_unstack = dados1.unstack()
dados_unstack

,1,2,3
a,-0.204708,0.478943,-0.519439
b,-0.555730,1.965781,1.393406
c,0.092908,0.281746,0.769023


In [ ]:
dados_unstack.loc[["a", "b"], 1]

a   -0.204708
b   -0.555730
Name: 1, dtype: float64

In [ ]:
# voltando para indice duplo. Operação inversa ao unstack
dados_stacked = dados_unstack.stack()
dados_stacked

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   2    1.965781
   3    1.393406
c  1    0.092908
   2    0.281746
   3    0.769023
dtype: float64

In [ ]:
# dois nivels de indices nas linhas e nas colunas
dados2 = pd.DataFrame(np.arange(12).reshape(4, 3), 
                      index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                      columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
dados2

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [ ]:
dados2.index.names = ["k1", "k2"]
dados2.columns.names = ["estados", "cor"]

In [ ]:
dados2

estados  Ohio     Colorado
cor     Green Red    Green
k1 k2                     
a  1        0   1        2
   2        3   4        5
b  1        6   7        8
   2        9  10       11

In [ ]:
# acessando uma coluna com o indice hierarquico
dados2["Colorado"]

cor    Green
k1 k2       
a  1       2
   2       5
b  1       8
   2      11

In [ ]:
dados2.loc[["a"], "Colorado"]          

cor    Green
k1 k2       
a  1       2
   2       5

In [ ]:
# alterando a ordem dos niveis hierarquicos das linhas
dados_swaplevel = dados2.swaplevel("k1", "k2")
dados_swaplevel

estados  Ohio     Colorado
cor     Green Red    Green
k2 k1                     
1  a        0   1        2
2  a        3   4        5
1  b        6   7        8
2  b        9  10       11

In [ ]:
# ordenando pelo indice
# dados2.swaplevel("k1", "k2").sort_index(level="k2")
dados_swaplevel.sort_index(level="k2")

estados  Ohio     Colorado
cor     Green Red    Green
k2 k1                     
1  a        0   1        2
   b        6   7        8
2  a        3   4        5
   b        9  10       11

In [ ]:
# estatíticas de nível de indices

# soma de todas as linhas com o indice k1
dados2.sum(level="k1")

estados  Ohio     Colorado
cor     Green Red    Green
k1                        
a           3   5        7
b          15  17       19

In [ ]:
dados2.sum(level="k2")

estados  Ohio     Colorado
cor     Green Red    Green
k2                        
1           6   8       10
2          12  14       16

In [ ]:
# soma no nivel de coluna, internamente está sendo feito um groupby
dados2.sum(level="cor", axis=1)

cor    Green  Red
k1 k2            
a  1       2    1
   2       8    4
b  1      14    7
   2      20   10

In [ ]:
# transformar colunas em indices hierarquicos
dados3 = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                       'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                       'd': [0, 1, 2, 0, 1, 2, 3]})
dados3

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [ ]:
# transformando colunas em indices hierarquicos
# drop=False não remove as colunas que foram usadas para criar os indices
dados_index = dados3.set_index(["c", "a"], drop=False)
dados_index

a  b    c  d
c   a              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 3  3  4  two  0
    4  4  3  two  1
    5  5  2  two  2
    6  6  1  two  3

## Combinando e mesclando
- merge (join): combina dados de df com base em uma ou mais chaves
- concat: empilha objetos ao longo de um eixo

### Merge (join)

In [ ]:
# merge
dados_left  = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'pastrami', 'corned beef', 'bacon', 'pastrami', 'honey ham', 'nova lox', 'comida vegana'],
                            'valor': [4, 3, 12, 6, 7.5, 8, 3, 5, 6, 5]})

dados_rigth = pd.DataFrame({'food': ['bacon', 'pulled pork', 'pastrami', 'corned beef', 'honey ham', 'nova lox'],
              'carne': ['pig', 'pig', 'cow', 'cow', 'pig',  'salmon' ]})

In [ ]:
dados_left

,food,valor
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0
9,comida vegana,5.0


In [ ]:
dados_rigth

,food,carne
0,bacon,pig
1,pulled pork,pig
2,pastrami,cow
3,corned beef,cow
4,honey ham,pig
5,nova lox,salmon


In [ ]:
# join pela coluna food para obter a nova coluna carne
# o padrão o join é inner, comida vegana não entra porque não tem correspondente no dados_rigth
pd.merge(dados_left, dados_rigth, on="food")

,food,valor,carne
0,bacon,4.0,pig
1,bacon,12.0,pig
2,bacon,8.0,pig
3,pulled pork,3.0,pig
4,pastrami,6.0,cow
5,pastrami,3.0,cow
6,corned beef,7.5,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [ ]:
# se as colunas envolvidas no join tiverem nomes diferentes
dados_left.rename(columns={"food": "left_food"}, inplace=True)
dados_rigth.rename(columns={"food": "rigth_food"}, inplace=True)

# definir cada none da tabela da esquerda e da direita
pd.merge(dados_left, dados_rigth, left_on="left_food", right_on="rigth_food")

,left_food,valor,rigth_food,carne
0,bacon,4.0,bacon,pig
1,bacon,12.0,bacon,pig
2,bacon,8.0,bacon,pig
3,pulled pork,3.0,pulled pork,pig
4,pastrami,6.0,pastrami,cow
5,pastrami,3.0,pastrami,cow
6,corned beef,7.5,corned beef,cow
7,honey ham,5.0,honey ham,pig
8,nova lox,6.0,nova lox,salmon


In [ ]:
# left join: incluir os registros da esquerda que não existem na direita
pd.merge(dados_left, dados_rigth, left_on='left_food', right_on='rigth_food', how='left').fillna("Esta comida não tem carne")

,left_food,valor,rigth_food,carne
0,bacon,4.0,bacon,pig
1,pulled pork,3.0,pulled pork,pig
2,bacon,12.0,bacon,pig
3,pastrami,6.0,pastrami,cow
4,corned beef,7.5,corned beef,cow
5,bacon,8.0,bacon,pig
6,pastrami,3.0,pastrami,cow
7,honey ham,5.0,honey ham,pig
8,nova lox,6.0,nova lox,salmon
9,comida vegana,5.0,Esta comida não tem carne,Esta comida não tem carne


In [ ]:
# merge com várias chaves
left2 = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})

right2 = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})

# passar mais de uma chave na clausula on=[]
pd.merge(left2, right2, on=['key1', 'key2'], how='right').fillna("Não possui valor")

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,bar,one,3,6
3,bar,two,Não possui valor,7


In [ ]:
# usando os índices como chaves de junção
left2.set_index(["key1", "key2"])
right2.set_index(["key1", "key2"])

pd.merge(left2, right2, how="inner", left_index=True, right_index=True, suffixes=("_left", "_right"))

,key1_left,key2_left,lval,key1_right,key2_right,rval
0,foo,one,1,foo,one,4
1,foo,two,2,foo,one,5
2,bar,one,3,bar,one,6


In [ ]:
# uma outra forma de fazer o join é usando a função de objeto .join
# essa forma funciona apenas em junção do indice

dados_left2  = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'pastrami', 'corned beef', 'bacon', 'pastrami', 'honey ham', 'nova lox', 'comida vegana'],
                            'valor': [4, 3, 12, 6, 7.5, 8, 3, 5, 6, 5]})

dados_right2 = pd.DataFrame({'food': ['bacon', 'pulled pork', 'pastrami', 'corned beef', 'honey ham', 'nova lox'],
              'carne': ['pig', 'pig', 'cow', 'cow', 'pig',  'salmon']})

# transformando a coluna food em chave
dados_left2.set_index("food", inplace=True)
dados_right2.set_index("food", inplace=True)

# dados_left2.join(dados_right2, on="food")
# quando não usa on= as chaves são automaticamente usadas para fazer o join
dados_left2.join(dados_right2)

,valor,carne
food,,
bacon,4.0,pig
bacon,12.0,pig
bacon,8.0,pig
comida vegana,5.0,NaN
corned beef,7.5,cow
honey ham,5.0,pig
nova lox,6.0,salmon
pastrami,6.0,cow
pastrami,3.0,cow


In [ ]:
# merge com um terceiro dataframe
dados_sabor = pd.DataFrame({'food': ['bacon', 'pulled pork', 'pastrami', 'corned beef', 'honey ham', 'nova lox', 'comida vegana'],
              'sabor': ['picante', 'picante', 'picante', 'suave', np.nan,  'doce', 'agridoce' ]})

dados_sabor.set_index("food", inplace=True)

# faz o join da tabela dados_left com dados_right e dados_sabor
# faz o tratamento dos valores np.nan nas colunas carne e sabor
dados_left2.join([dados_right2, dados_sabor]).fillna({"carne": "SEM CARNE", "sabor": "SEM SABOR"})

,valor,carne,sabor
food,,,
bacon,4.0,pig,picante
bacon,12.0,pig,picante
bacon,8.0,pig,picante
comida vegana,5.0,SEM CARNE,agridoce
corned beef,7.5,cow,suave
honey ham,5.0,pig,SEM SABOR
nova lox,6.0,salmon,doce
pastrami,6.0,cow,picante
pastrami,3.0,cow,picante


### Concatenando (union)

In [ ]:
# empilha a s1 com a s2 union
s1 = pd.Series(np.arange(1, 6, 1))
s2 = pd.Series(np.arange(6, 11, 1))

# por padrão a concatenação é feita no eixo 0
pd.concat([s1, s2])

0     1
1     2
2     3
3     4
4     5
0     6
1     7
2     8
3     9
4    10
dtype: int64

In [ ]:
# concatenação no eixo 1
s3 = pd.concat([s1, s2], axis=1)
s3

,0,1
0,1,6
1,2,7
2,3,8
3,4,9
4,5,10


In [ ]:
# remove a linha 4 para ver como o join irá funcionar
s3.drop([4], axis=0, inplace=True)

# o join com a s3 faz a linha 4 de s1 sumir
# linha 4 fica com np.nan porque o padrão é left join
pd.concat([s1, s3], axis=1)

,0,0,1
0,1,1.0,6.0
1,2,2.0,7.0
2,3,3.0,8.0
3,4,4.0,9.0
4,5,NaN,NaN


In [ ]:
# por padrão funciona com left, para forçar um inner usar
# a linha 4 some
pd.concat([s1, s3], axis=1, join="inner")

,0,0,1
0,1,1,6
1,2,2,7
2,3,3,8
3,4,4,9


## Pivot
- stack: colunas para linhas
- unstack: linhas para colunas

In [ ]:
dados4 = pd.DataFrame(np.arange(2 * 3).reshape((2, 3)), 
                      index=pd.Index(["Brasil", "Argentina"], name="Paises"),
                      columns=pd.Index(["Um", "Dois", "Tres"], name="Cabecalho"))

dados4

Cabecalho,Um,Dois,Tres
Paises,,,
Brasil,0,1,2
Argentina,3,4,5


In [ ]:
# colunas para linhas. Neste caso cria um multiindex
resultado_stack = dados4.stack()
resultado_stack

Paises     Cabecalho
Brasil     Um           0
           Dois         1
           Tres         2
Argentina  Um           3
           Dois         4
           Tres         5
dtype: int64

In [ ]:
# de linhas para colunas
# por padrão sempre o nível mais interno é desempilhado ou empilhado
resultado_unstack = resultado_stack.unstack()
resultado_unstack

Cabecalho,Um,Dois,Tres
Paises,,,
Brasil,0,1,2
Argentina,3,4,5


In [ ]:
# level= Escolher o nível mais externo para desempilhar
# Por padrão o nível interno é desempilhado, para escolher qual nível desempilhar, usar level
resultado_unstack = resultado_stack.unstack(level="Paises")
resultado_unstack

Paises,Brasil,Argentina
Cabecalho,,
Um,0,3
Dois,1,4
Tres,2,5


## Unpivot

In [ ]:
# cria um df com uma coluna fixa e duas colunas que serão unpivoteadas
dados_pivot  = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'pastrami', 'corned beef', 'bacon', 'pastrami', 'honey ham', 'nova lox', 'comida vegana'],
                            'NoPrazo': [4, 3, 12, 6, 7.5, 8, 3, 5, 6, 5],
                            'Vencido': [1, 1, 1, 2, 5, 2, 3, 3, 1, 1]})

# coluna fixa é transformada em indice
dados_pivot.set_index(["food"], inplace=True)

# nome para os rótulos de coluna e indice
dados_pivot.columns.name = "Situacao"
dados_pivot.index.name = "food"

# transforma as colunas NoPrazo e Situação em indices de linhas
#dados_pivot.stack()

# recria um indice padrão de inteiros iniciando em 0
# dados_pivot.stack().reset_index()

# No final os valores perdem o rotulo e ficam com o valor 0, então renomeamos
dados_unpivot = dados_pivot.stack().reset_index().rename(columns={0: "quantidade"})
dados_unpivot

,food,Situacao,quantidade
0,bacon,NoPrazo,4.0
1,bacon,Vencido,1.0
2,pulled pork,NoPrazo,3.0
3,pulled pork,Vencido,1.0
4,bacon,NoPrazo,12.0
5,bacon,Vencido,1.0
6,pastrami,NoPrazo,6.0
7,pastrami,Vencido,2.0
8,corned beef,NoPrazo,7.5
9,corned beef,Vencido,5.0


In [ ]:
# voltado para o formato anterior

# dados_pivot = dados_unpivot.pivot("food", "Situacao", "quantidade")

# agrupa as linhas para remover as duplicidades e agregar os valores
dados_unpivot_agregados = dados_unpivot.groupby(by=["food", "Situacao"]).sum("quantidade")

# redefine o indice para que as colunas agrupadas sejam tratadas como colunas e não como indices
dados_unpivot_agregados.reset_index(inplace=True)

# dispivoteia o troço todo
dados_unpivot_agregados = dados_unpivot_agregados.pivot("food", "Situacao", "quantidade")
dados_unpivot_agregados

Situacao,NoPrazo,Vencido
food,,
bacon,24.0,4.0
comida vegana,5.0,1.0
corned beef,7.5,5.0
honey ham,5.0,3.0
nova lox,6.0,1.0
pastrami,9.0,5.0
pulled pork,3.0,1.0


## Melt
Transforma colunas em linhas


In [ ]:
dados_melt = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                          'A': [1, 2, 3],
                          'B': [4, 5, 6],
                          'C': [7, 8, 9]})
dados_melt

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [ ]:
# todas as colunas exceto "key" são trasformadas em linhas.
# são criadas os rotulos de coluna "variable" e "value"
# todas as colunas se encontram na coluna "variable"
dados_melt2 = pd.melt(dados_melt, ["key"])
dados_melt2

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [ ]:
# por padrão melt usa todas as colunas "A", "B", "C" para se transformar em linhas, mas podemos especificar um subconjunto
# faz melt apenas das colunas A e B ignora a C
dados_melt3 = pd.melt(dados_melt, ["key"], value_vars=["A", "B"])
dados_melt3

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [ ]:
# retona para o formato anterior com todas as 3 colunas
dados_melt2 = dados_melt2.pivot("key", "variable", "value").reset_index()
dados_melt2

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [ ]:
# retorna com apenas A e B
dados_melt3 = dados_melt3.pivot("key", "variable", "value").reset_index()
dados_melt3

variable,key,A,B
0,bar,2,5
1,baz,3,6
2,foo,1,4
